In [1]:
from datetime import datetime as dt
import re
import os
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Overview - Using Selenium to scrape Florida Special Election winners from 1996-current
Done after scraping general election winners, uses the same framework

# Setup
#### Setting up driver with download path/options

In [5]:
# Set up Chrome options for automated downloads
chrome_options = Options()
download_path = ""  # put what is relevant for you
# originally set custom download path, but defaulted to download file to reduce unneccesary bug fixing
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(options=chrome_options)

#### Loading initial URL, switching to the correct frame

In [8]:
driver.get("https://results.elections.myflorida.com/Specials.asp")
driver.switch_to.frame("WinRaces")

#### Getting list of relevant elections from drop down element

In [14]:
dropdown_element = Select(driver.find_element(By.NAME, 'SelectElection'))

list_of_elections = [option.text for option in dropdown_element.options]
relevant_elections = list_of_elections[1:]


# Main Loop
#### Goes through each election option above by
1. Selecting next relevant election in drop down menu
2. Navigating to each election's relevant Download page
3. Downloading and renaming the file to be easier to read
4. Resetting the driver to the election selection page to prepare for the next loop

In [16]:
for election in relevant_elections:
    
    dropdown_element.select_by_visible_text(election)
    
    
    #updating the driver to the new URL and finding relevant elements
    driver.get(driver.current_url)
    time.sleep(2)
    driver.switch_to.frame("WinRaces")
    dropdown_element = Select(driver.find_element(By.NAME, 'SelectElection'))
    partial_link = driver.find_element(By.PARTIAL_LINK_TEXT, "Download")
    
    #clicking "Donwload" to go to each election's download page, updating driver to new window
    partial_link.click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])
    download_button = driver.find_element(By.CSS_SELECTOR, 'input[type="SUBMIT"][name="FormsButton2"][value="Download"]')
    #download_button.click()
    
  
    #extracting relevant information from substring to determine standard download name and replacement download name
        #website downloads all files by naming them by corresponding election date ex: '11082022Election.txt'
    election_date_url = driver.current_url

    match = re.search(r"ElectionDate=([^&]*)&DATA", election_date_url)
    if match:
        election_date = match.group(1)
    else:
        print("No match found")
        print("No match found")
            # Reformatting the  to MM/DD/YYYY with leading zeros
    date_obj = dt.strptime(election_date, "%m/%d/%Y")
    standardized_date = date_obj.strftime("%m/%d/%Y")
    og_file_name = standardized_date.replace("/","") + "Election.txt"
    new_file_name = election.replace(" - ",'_').replace(" ",'_').replace("&","and").replace(";","").replace(":","")
    new_file_name = new_file_name.replace("/","_").replace(',','-').replace("__",'-')
    new_file_name = new_file_name.replace("__","_") + ".txt"

      #downloading and renaming file

    download_button.click()
    time.sleep(2)
    original_file = os.path.join(download_path, og_file_name)  #original file that will be replaced
    new_file = os.path.join(download_path, new_file_name)

    if os.path.exists(original_file):  ## replacing old file name with proper file name
        os.rename(original_file, new_file)
    else:
        print("Download failed or file not found.")
    #closing download tab and updating to elections tab
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("WinRaces")
    dropdown_element = Select(driver.find_element(By.NAME,'SelectElection'))


In [18]:
driver.close()